In [1]:
import numpy as np
import random

In [2]:
def distance(pos1,pos2):
    distance = 0
    for i in range(len(pos1)):
        distance += (pos1[i]-pos2[i])**2
    return (distance)**0.5

In [3]:
distance((4,5),(4,6))

1.0

In [4]:
class Game:
    def __init__(self, size):
        self.board = np.zeros(shape=(size,size))
        self.team = [[],[]]
        self.index = [0,0]

    def add_unit(self, unit, position, team):
        if self.board[position] != 0:
            return
        self.board[position] = unit*[1,-1][team]
        self.team[team].append(position)

        
# Tank units move like infantry units (towards closest enemy), except their kill percentage is halfed, 
# and their death percentage is halfed (death percentage to be implemented later).
    def tank_unit_ai(self, position):
        team = self.board[position] < 0
        
        # MOVE PHASE: will move towards the closest enemy
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        enemy_pos = self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        if enemy_pos[0]!=position[0]:
            new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        elif enemy_pos[1]!=position[1]: 
            new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        else:
            new_pos = position
        if self.board[new_pos] != 0:
            new_pos = position
        else:
            self.board[position] = 0
            # tank unit will be displayed on the board by number 3
            self.board[new_pos] = [1,-1][team]*3
        
        # ACTION PHASE: killed smth, didn't kill smth, or can't kill smth
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # the closest enemy to the tank unit
        min_dist = min(enemy)
        if min_dist <= 1.1:
    # this just figures out which unit should be killed and the actual killing happens later on
    # kill percentage is half for tank
            if random.random() > 0.6:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    
    def default_ai(self, position):
        team = self.board[position] < 0
        # Move phase
        # nearest enemy troop
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        enemy_pos = self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        if enemy_pos[0]!=position[0]:
            new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        elif enemy_pos[1]!=position[1]: 
            new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        else:
            new_pos = position
        if self.board[new_pos] != 0:
            # don't move
            # print('cant move', self.board[new_pos], new_pos)
            new_pos = position
        else:
            self.board[position] = 0
            self.board[new_pos] = [1,-1][team]*1
        
        # action phase
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # print('enemy distances', enemy)
        min_dist = min(enemy)
        if min_dist <= 1.1:
            # print(team,'attempts to kill someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
            if random.random() > 0.2:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    def turn(self, team):
        unit_ai = [0,self.default_ai]
        if self.index[team] == len(self.team[team]):
            if self.index[team-1] == len(self.team[team-1]):
                self.index = [0,0]
            else:
                return -1
        unit = int(abs(self.board[self.team[team][self.index[team]]]))
        # print('index',self.index[team], 'position',self.team[team][self.index[team]], 'team', team)
        new_position, death = unit_ai[unit](self.team[team][self.index[team]])
        self.team[team][self.index[team]] = new_position
        if death != None:
            self.board[death] = 0
            enemy_index = 0
            for i in range(len(self.team[team-1])):
                if death == self.team[team-1][i]:
                    enemy_index = i
                    break
            if self.index[team-1]>=enemy_index:
                self.index[team-1]-=1
            self.team[team-1].pop(enemy_index)
            # print(self.team[team-1], 'is left over')
            if len(self.team[team-1]) == 0:
                return team
        self.index[team]+=1
        return -1
    
    def run(self):
        turns = 0
        while(1):
            for team in [0,1]:
                turns += 1
                win = self.turn(team)
                if win != -1:
                    return win, turns
                
    def create_board(self, board):
        pieces = np.transpose(np.nonzero(board))
        for piece in pieces:
            piece = tuple(piece)
            # print(piece)
            self.add_unit(abs(board[piece]), piece, board[piece]<0)

        


    

    


In [5]:
size = 10
board = np.zeros((size, size))
board[1,2] = 1
board[6,1] = 1
board[4,2] = 1
board[6,4] = -1
board[6,5] = -1
board[5,5] = -1
print(board)
winners = []
for i in range(1000):
    game = Game(size)
    game.create_board(board)
    winners.append(game.run())
print(np.mean(winners, axis=0))

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0. -1. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


<ipython-input-4-97afd1940f74>:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.board[position] = unit*[1,-1][team]
<ipython-input-4-97afd1940f74>:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.team[team].append(position)
<ipython-input-4-97afd1940f74>:26: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
<ipython-input-4-97afd1940f74>:37: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.board[new_pos] = [1,-1][team]*1
<ipython-input-4-97afd1940f74>:28: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])


[ 0.731 31.793]
